In [45]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## print all of text
pd.options.display.max_colwidth = 1000

# 1. Example 1: no credentials; no wrapper (API for NAEP data)

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))



<Response [200]>

<class 'requests.models.Response'>


In [4]:
## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

In [5]:

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [6]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [7]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json()

<Response [200]>

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [8]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except:
    pass

### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [9]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along thos elines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1953\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1713\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 560\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 136"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Example to go through as group: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [10]:
def pull_one_grade(one_grade):
    
    ## first construct the query 
    q = (
    'https://www.nationsreportcard.gov/'
    'Dataservice/GetAdhocData.aspx?'
    'type=data&subject=writing&grade='+str(one_grade)+'&'+
    'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
    
    ## then, use requests to call
    resp = requests.get(q)
    
    ## next, transform into json
    try:
        resp_j = resp.json()
        resp_df = pd.DataFrame(resp_j['result'])
        return(resp_df)
    except:
        pass  


all_grades = [pull_one_grade(grade) for grade in [4, 8, 12]]
all_grades_df = pd.concat(all_grades)
all_grades_df


,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,GENDER,Gender,2,Female,158.567105,1,0
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper (API for Yelp)

In [14]:
os.getcwd()

'/Users/rebeccajohnson/Dropbox/PPOL564_slides_activities/activities/fall_22/solutions'

In [15]:
## load creds- STUDENTS: REPLACE WITH PATHNAME TO YOURS 
with open("../../../cred.yml", 'r') as stream:
    try:
        creds = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

## get the key
API_KEY = creds['yelp_api']['api_key']

In [16]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Washington,DC,20057"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()
#yelp_genjson

<Response [200]>

In [17]:
## example business
yelp_genjson['businesses'][0]




{'id': '30PcKI8-eYijw4hn-ypUIg',
 'alias': 'brasserie-liberte-washington',
 'name': 'Brasserie Liberte',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/UtYuFQ9tXhMyk15rsxThyg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/brasserie-liberte-washington?adjust_creative=ABQTB3e9fTiSiyqs0c-3Bg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ABQTB3e9fTiSiyqs0c-3Bg',
 'review_count': 301,
 'categories': [{'alias': 'french', 'title': 'French'},
  {'alias': 'brasseries', 'title': 'Brasseries'}],
 'rating': 4.0,
 'coordinates': {'latitude': 38.906176, 'longitude': -77.064069},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3251 Prospect St NW',
  'address2': None,
  'address3': '',
  'city': 'Washington, DC',
  'zip_code': '20007',
  'country': 'US',
  'state': 'DC',
  'display_address': ['3251 Prospect St NW', 'Washington, DC 20007']},
 'phone': '+12028788404',
 'display_phone': '(202) 878-8404',
 'distance': 1067.43939860

In [18]:
## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,30PcKI8-eYijw4hn-ypUIg,brasserie-liberte-washington,Brasserie Liberte,https://s3-media2.fl.yelpcdn.com/bphoto/UtYuFQ...,False,https://www.yelp.com/biz/brasserie-liberte-was...,301,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,"{'latitude': 38.906176, 'longitude': -77.064069}",[delivery],$$,"{'address1': '3251 Prospect St NW', 'address2'...",+12028788404,(202) 878-8404,1067.439399
1,h7ZMmxZq5DXtvxzm_TU3Mg,sticx-stonz-dc-washington,Sticx + Stonz DC,https://s3-media1.fl.yelpcdn.com/bphoto/yShM5g...,False,https://www.yelp.com/biz/sticx-stonz-dc-washin...,21,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",5.0,"{'latitude': 38.91445, 'longitude': -77.06723}",[],NaN,"{'address1': '1728 Wisconsin Ave NW', 'address...",,,916.802336
2,sXFBW4wWKmAraO4O_Z6nWg,call-your-mother-washington-4,Call Your Mother,https://s3-media3.fl.yelpcdn.com/bphoto/X6vaJe...,False,https://www.yelp.com/biz/call-your-mother-wash...,217,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 38.9076174, 'longitude': -77.068837}",[delivery],$$,"{'address1': '3428 O St NW', 'address2': '', '...",+12028533653,(202) 853-3653,610.982885
3,9OnxdYEAyivZnqQhFsYEJA,lutèce-washington-2,Lutèce,https://s3-media1.fl.yelpcdn.com/bphoto/OLK1pB...,False,https://www.yelp.com/biz/lut%C3%A8ce-washingto...,96,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 38.909408, 'longitude': -77.0646}",[],NaN,"{'address1': '1522 Wisconsin Ave', 'address2':...",+12023338830,(202) 333-8830,947.741226
4,IMUlQlXTqgsWTUZLifuRZQ,clydes-of-georgetown-washington-3,Clyde's of Georgetown,https://s3-media1.fl.yelpcdn.com/bphoto/nHMLPK...,False,https://www.yelp.com/biz/clydes-of-georgetown-...,1361,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 38.9049110412598, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '3236 M St NW', 'address2': '', '...",+12023339180,(202) 333-9180,1102.897356


In [19]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
30PcKI8-eYijw4hn-ypUIg,30PcKI8-eYijw4hn-ypUIg,brasserie-liberte-washington,Brasserie Liberte,https://s3-media2.fl.yelpcdn.com/bphoto/UtYuFQ...,https://www.yelp.com/biz/brasserie-liberte-was...,$$,+12028788404,(202) 878-8404
h7ZMmxZq5DXtvxzm_TU3Mg,h7ZMmxZq5DXtvxzm_TU3Mg,sticx-stonz-dc-washington,Sticx + Stonz DC,https://s3-media1.fl.yelpcdn.com/bphoto/yShM5g...,https://www.yelp.com/biz/sticx-stonz-dc-washin...,NaN,,
sXFBW4wWKmAraO4O_Z6nWg,sXFBW4wWKmAraO4O_Z6nWg,call-your-mother-washington-4,Call Your Mother,https://s3-media3.fl.yelpcdn.com/bphoto/X6vaJe...,https://www.yelp.com/biz/call-your-mother-wash...,$$,+12028533653,(202) 853-3653
9OnxdYEAyivZnqQhFsYEJA,9OnxdYEAyivZnqQhFsYEJA,lutèce-washington-2,Lutèce,https://s3-media1.fl.yelpcdn.com/bphoto/OLK1pB...,https://www.yelp.com/biz/lut%C3%A8ce-washingto...,NaN,+12023338830,(202) 333-8830
IMUlQlXTqgsWTUZLifuRZQ,IMUlQlXTqgsWTUZLifuRZQ,clydes-of-georgetown-washington-3,Clyde's of Georgetown,https://s3-media1.fl.yelpcdn.com/bphoto/nHMLPK...,https://www.yelp.com/biz/clydes-of-georgetown-...,$$,+12023339180,(202) 333-9180
YvqJqlX5HtCtgFbd3KEV3w,YvqJqlX5HtCtgFbd3KEV3w,1789-restaurant-and-bar-washington,1789 Restaurant & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/k7li2X...,https://www.yelp.com/biz/1789-restaurant-and-b...,$$$,+12029651789,(202) 965-1789
hvtH8uV4-jpm1Ki6c72XWw,hvtH8uV4-jpm1Ki6c72XWw,the-sovereign-washington-3,The Sovereign,https://s3-media1.fl.yelpcdn.com/bphoto/SOWzcJ...,https://www.yelp.com/biz/the-sovereign-washing...,$$,+12027745875,(202) 774-5875


# Activity for you: pull restaurants in a different location

- Try running a business search query for another place in the United States by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://www.yelp.com/developers/documentation/v3/business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a single business ID as an input, (2) calls the reviews API for that id, (3) returns the reviews. Execute that function over all the ids from your first pull and print the results as a dataframe


In [23]:
## query for another location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Burlington,VT,08540"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header).json()
yelp_genresp_list = [clean_yelp_json(one_b) for one_b in yelp_genresp['businesses']]
yelp_stronly_df = pd.concat(yelp_genresp_list)
yelp_stronly_df.head()

hen_id = yelp_stronly_df.id.iloc[0]
#hen_id



,id,alias,name,image_url,url,price,phone,display_phone
kuyCA_1tbf6kSijRB6qcXA,kuyCA_1tbf6kSijRB6qcXA,hen-of-the-wood-burlington-burlington-4,Hen of the Wood - Burlington,https://s3-media2.fl.yelpcdn.com/bphoto/8Dm0SN...,https://www.yelp.com/biz/hen-of-the-wood-burli...,$$$,+18025400534,(802) 540-0534
-ASe09az4o1j4z9Y628ejg,-ASe09az4o1j4z9Y628ejg,the-farmhouse-tap-and-grill-burlington,The Farmhouse Tap & Grill,https://s3-media3.fl.yelpcdn.com/bphoto/8HDOqV...,https://www.yelp.com/biz/the-farmhouse-tap-and...,$$,+18028590888,(802) 859-0888
sh9sPakaQqVYc3hOtV-x6g,sh9sPakaQqVYc3hOtV-x6g,the-gryphon-burlington,The Gryphon,https://s3-media1.fl.yelpcdn.com/bphoto/tztWXN...,https://www.yelp.com/biz/the-gryphon-burlingto...,$$,+18024895699,(802) 489-5699
IeRVd34QMvQAVNtPpY9ImQ,IeRVd34QMvQAVNtPpY9ImQ,honey-road-burlington,Honey Road,https://s3-media3.fl.yelpcdn.com/bphoto/w6bxNf...,https://www.yelp.com/biz/honey-road-burlington...,$$,+18024972145,(802) 497-2145
rrLmE9myyWjA-ZzpzfHUIg,rrLmE9myyWjA-ZzpzfHUIg,juniper-bar-and-restaurant-burlington,Juniper Bar & Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/0afoOt...,https://www.yelp.com/biz/juniper-bar-and-resta...,$$,+18026510080,(802) 651-0080


In [24]:
## query to get reviews for a specific id
reviews_base = "https://api.yelp.com/v3/businesses/"
reviews_query = ('{base_url}'
                '{yelp_id}'
                '/reviews').format(base_url = reviews_base,
                yelp_id = hen_id)
reviews_query

reviews_j = requests.get(reviews_query, headers = header).json()
reviews_j['reviews']

'https://api.yelp.com/v3/businesses/kuyCA_1tbf6kSijRB6qcXA/reviews'

[{'id': '8Dn5jEjnXAeIFUkJJHMdbA',
  'url': 'https://www.yelp.com/biz/hen-of-the-wood-burlington-burlington-4?adjust_creative=ABQTB3e9fTiSiyqs0c-3Bg&hrid=8Dn5jEjnXAeIFUkJJHMdbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=ABQTB3e9fTiSiyqs0c-3Bg',
  'text': "They don't serve THE MUSHROOM TOAST ANYMORE! I drove 5h from NY to here, just for the mushroom toast.",
  'rating': 4,
  'time_created': '2022-10-14 16:11:50',
  'user': {'id': 'r4iww6fRa9heJ5BW6VT1GQ',
   'profile_url': 'https://www.yelp.com/user_details?userid=r4iww6fRa9heJ5BW6VT1GQ',
   'image_url': None,
   'name': 'Fangzhong X.'}},
 {'id': 'HAyZ_D9RIUVezH4Zuo45AQ',
  'url': 'https://www.yelp.com/biz/hen-of-the-wood-burlington-burlington-4?adjust_creative=ABQTB3e9fTiSiyqs0c-3Bg&hrid=HAyZ_D9RIUVezH4Zuo45AQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=ABQTB3e9fTiSiyqs0c-3Bg',
  'text': 'Holy moly 10/10! My husband and I own a restaurant just north of Boston and can be hard to impr

In [ ]:
## challenge exercise:
## translate that into a function

In [34]:
def get_username(one_review):
    username = one_review['user']['name']
    return(username)

In [42]:
def pull_reviews(one_id, header):
    
    reviews_base = "https://api.yelp.com/v3/businesses/"
    reviews_query = ('{base_url}'
                '{yelp_id}'
                '/reviews').format(base_url = reviews_base,
                yelp_id = one_id)
    reviews_j = requests.get(reviews_query, headers = header).json()
    reviews_fordf = reviews_j['reviews']
    users = [get_username(one_review) for one_review in reviews_fordf]
    all_reviews = pd.DataFrame(reviews_fordf)
    all_reviews['username'] = users
    all_reviews.drop(columns = 'user', inplace = True)
    return(all_reviews)
    

In [47]:
reviews_multbiz_list = [pull_reviews(one_id = one_id, header = header) for one_id in 
                    yelp_stronly_df.id]

reviews_multbiz_df = pd.concat(reviews_multbiz_list)

reviews_multbiz_df[['text', 'id']].head()

## can merge on name from original api call

,text,id
0,"They don't serve THE MUSHROOM TOAST ANYMORE! I drove 5h from NY to here, just for the mushroom toast.",8Dn5jEjnXAeIFUkJJHMdbA
1,Holy moly 10/10! My husband and I own a restaurant just north of Boston and can be hard to impress but the Hen did it! \n\nOur server was awesome. We forget...,HAyZ_D9RIUVezH4Zuo45AQ
2,"I agree with others here that this restaurant is not worth the hype. I was expecting to be wowed, both due to reports from co-workers who've dined here and...",R5e2AXPv0VrhqgnXTRfz-Q
0,My husband and I had a short period of time last weekend to grab some dinner before heading to the airport. When we arrived the wait was about 30-45min but...,WwN0CIbHCXz759gQ9dA4Cw
1,"Shortly after we arrived in Burlington, we were hungry and in search of food and glad we found The Farmhouse Tap & Grill. Since there was a long wait to...",uGqBIrx2NH98n3j-VSElog
